# 🔍 Cybercrime Intelligence Newspaper Agent

This notebook implements a simple autonomous AI agent that monitors news sources for cybercrime-related incidents. The agent automatically fetches newspaper articles, identifies fraud and cybercrime mentions, summarizes key information, and generates a daily intelligence brief for law enforcement agencies. This is a **Stage-1 prototype** focused on clarity and simplicity—designed to evolve into a production-ready system.

## What is an AI Agent?

Think of an AI agent as a **smart assistant that can act on its own**. Unlike a simple chatbot that just answers questions, an agent:

1. **Perceives** its environment (reads news, checks data)
2. **Thinks** about what it sees (analyzes, filters, decides)
3. **Acts** on its analysis (summarizes, reports, alerts)
4. **Repeats** this cycle autonomously

In Ed Donner's framework, an agent has:
- **Goals** (find cybercrime news)
- **Tools** (web scraping, LLM reasoning)
- **Memory** (what it has already processed)
- **Actions** (generate reports)

Our agent is simple but powerful: it replaces hours of manual news monitoring with an automated, intelligent system.

In [ ]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re

# For LLM integration - using Ollama with local Llama 3.2 model
try:
    import ollama
    # Test if Ollama is running and model is available
    try:
        ollama.chat(model='llama3.2:latest', messages=[{'role': 'user', 'content': 'test'}])
        LLM_AVAILABLE = True
        print("✅ Ollama connected successfully with llama3.2:latest model")
    except Exception as e:
        print(f"⚠️ Ollama error: {e}")
        print("   Make sure Ollama is running and llama3.2:latest model is downloaded")
        print("   Run: ollama pull llama3.2:latest")
        LLM_AVAILABLE = False
except ImportError:
    print("⚠️ Ollama library not installed. Install with: pip install ollama")
    print("   Will use mock summaries for demo.")
    LLM_AVAILABLE = False

print("\n✅ Libraries imported successfully")
print(f"📅 Today's date: {datetime.now().strftime('%Y-%m-%d')}")

## The Agent Loop Explained

Our agent follows a simple four-step cycle:

### 1️⃣ **FETCH** (Perception)
- Retrieve news headlines from RSS feeds from major Indian news sources
- Parse the content into structured data

### 2️⃣ **THINK** (Reasoning)
- Filter articles using cybercrime keywords
- Identify which news items are relevant to law enforcement

### 3️⃣ **ACT** (Decision Making)
- Use an LLM to summarize relevant articles
- Extract key insights and implications

### 4️⃣ **REPORT** (Output)
- Generate a clean, actionable intelligence brief
- Present findings in a structured format

This is the **core pattern** of agentic AI—simple but infinitely scalable.

In [ ]:
# STEP 1: FETCH - Newspaper Article Fetcher

def fetch_from_rss(rss_url, source_name):
    """
    Fetch articles from an RSS feed.
    
    Args:
        rss_url: URL of the RSS feed
        source_name: Name of the news source
    
    Returns:
        List of article dictionaries
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(rss_url, timeout=15, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'xml')
        items = soup.find_all('item')
        
        articles = []
        for item in items[:20]:  # Limit to 20 articles per source
            title = item.title.text.strip() if item.title else ''
            description = item.description.text.strip() if item.description else ''
            link = item.link.text.strip() if item.link else ''
            
            # Clean HTML tags from description if present
            if description:
                description = BeautifulSoup(description, 'html.parser').get_text()
            
            if title:  # Only add if we have at least a title
                articles.append({
                    'title': title,
                    'summary': description if description else title,
                    'source': source_name,
                    'url': link
                })
        
        print(f"   ✓ {source_name}: {len(articles)} articles")
        return articles
    
    except Exception as e:
        print(f"   ✗ {source_name}: Error - {str(e)[:50]}")
        return []


def fetch_news_articles():
    """
    Fetch news headlines from multiple Indian news sources via RSS feeds.
    
    Returns: List of article dictionaries with title, summary, source, and url
    """
    
    # Major Indian news sources with RSS feeds
    rss_feeds = [
        # Times of India
        ('https://timesofindia.indiatimes.com/rssfeedstopstories.cms', 'Times of India'),
        
        # The Hindu
        ('https://www.thehindu.com/news/national/feeder/default.rss', 'The Hindu'),
        
        # NDTV
        ('https://feeds.feedburner.com/ndtvnews-top-stories', 'NDTV'),
        
        # India Today
        ('https://www.indiatoday.in/rss/1206514', 'India Today'),
        
        # Hindustan Times
        ('https://www.hindustantimes.com/feeds/rss/india-news/rssfeed.xml', 'Hindustan Times'),
        
        # Indian Express
        ('https://indianexpress.com/feed/', 'Indian Express'),
    ]
    
    all_articles = []
    
    print("📰 Fetching articles from news sources...")
    
    for rss_url, source_name in rss_feeds:
        articles = fetch_from_rss(rss_url, source_name)
        all_articles.extend(articles)
    
    print(f"\n📊 Total articles fetched: {len(all_articles)}")
    return all_articles


# Execute the fetch
all_articles = fetch_news_articles()
print(f"\n✅ Stage 1 (FETCH) complete: {len(all_articles)} articles retrieved")

In [ ]:
# STEP 2: THINK - Filter for Cybercrime Content

def filter_cybercrime_articles(articles):
    """
    Filter articles for cybercrime-related keywords.
    
    This is a simple keyword-based approach.
    In Stage-2, we could use LLM-based classification for better accuracy.
    
    Args:
        articles: List of article dictionaries
    
    Returns:
        List of filtered articles related to cybercrime
    """
    
    # Cybercrime-related keywords (government intelligence focus)
    keywords = [
        'fraud', 'scam', 'cybercrime', 'cyber crime', 'digital arrest',
        'deepfake', 'phishing', 'hacking', 'ransomware', 'data breach',
        'online fraud', 'upi fraud', 'cryptocurrency scam', 'ponzi scheme',
        'identity theft', 'cyber attack', 'malware', 'fake website',
        'cyber security', 'cybersecurity', 'hacker', 'cyber threat',
        'online scam', 'digital fraud', 'bank fraud', 'credit card fraud'
    ]
    
    filtered = []
    
    for article in articles:
        # Combine title and summary for keyword matching
        text = (article['title'] + ' ' + article['summary']).lower()
        
        # Check if any keyword appears in the text
        for keyword in keywords:
            if keyword in text:
                article['matched_keyword'] = keyword  # Track what triggered the match
                filtered.append(article)
                break  # One match is enough
    
    print(f"🎯 Filtered to {len(filtered)} cybercrime-related articles")
    print(f"   (Removed {len(articles) - len(filtered)} irrelevant articles)")
    
    return filtered


# Execute the filtering
cybercrime_articles = filter_cybercrime_articles(all_articles)

# Show what we found
print("\n📋 Cybercrime articles identified:")
for idx, article in enumerate(cybercrime_articles, 1):
    print(f"   {idx}. {article['title'][:60]}...")
    print(f"      Matched keyword: '{article['matched_keyword']}'")

print(f"\n✅ Stage 2 (THINK) complete: {len(cybercrime_articles)} relevant articles")

In [ ]:
# STEP 3: ACT - LLM Summarization and Analysis

def summarize_with_llm(article):
    """
    Use an LLM to create intelligence-focused summaries.
    
    The LLM extracts:
    - Key facts (who, what, where, when)
    - Law enforcement implications
    - Actionable insights
    
    Args:
        article: Dictionary with article data
    
    Returns:
        Dictionary with intelligence summary
    """
    
    if not LLM_AVAILABLE:
        # Fallback: Return a basic summary without LLM
        return {
            'title': article['title'],
            'summary': article['summary'][:200] + '...' if len(article['summary']) > 200 else article['summary'],
            'implications': 'LLM not available - using basic summary',
            'source': article['source'],
            'url': article.get('url', 'N/A')
        }
    
    # Create a focused prompt for intelligence analysis
    prompt = f"""
You are an intelligence analyst for a government cybercrime unit.

Article Title: {article['title']}
Article Content: {article['summary']}
Source: {article['source']}

Provide a brief intelligence summary (2-3 sentences) covering:
1. Key facts (who, what, amounts involved)
2. Why this matters for law enforcement
3. Any patterns or trends

Keep it concise and actionable.
"""
    
    try:
        # Use Ollama with local Llama 3.2 model
        response = ollama.chat(
            model='llama3.2:latest',
            messages=[
                {
                    'role': 'system',
                    'content': 'You are a cybercrime intelligence analyst. Provide concise, factual analysis.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ],
            options={
                'temperature': 0.3,  # Lower temperature for factual analysis
                'num_predict': 200   # Limit response length
            }
        )
        
        intelligence_summary = response['message']['content'].strip()
        
        return {
            'title': article['title'],
            'summary': intelligence_summary,
            'implications': 'Analyzed by Llama 3.2 (local)',
            'source': article['source'],
            'url': article.get('url', 'N/A')
        }
    
    except Exception as e:
        print(f"⚠️ LLM error for article '{article['title'][:30]}...': {str(e)[:50]}")
        return {
            'title': article['title'],
            'summary': article['summary'][:200] + '...' if len(article['summary']) > 200 else article['summary'],
            'implications': 'LLM analysis failed - using original summary',
            'source': article['source'],
            'url': article.get('url', 'N/A')
        }


# Process all filtered articles
intelligence_summaries = []

print("🤖 Analyzing articles with Llama 3.2 (local model)...\n")

for idx, article in enumerate(cybercrime_articles, 1):
    print(f"   Processing {idx}/{len(cybercrime_articles)}: {article['title'][:50]}...")
    summary = summarize_with_llm(article)
    intelligence_summaries.append(summary)

print(f"\n✅ Stage 3 (ACT) complete: {len(intelligence_summaries)} summaries generated")

In [ ]:
# STEP 4: REPORT - Generate Daily Intelligence Brief

def generate_intelligence_report(summaries):
    """
    Generate a clean, actionable daily intelligence brief.
    
    This is what gets delivered to law enforcement analysts.
    """
    
    report_date = datetime.now().strftime('%Y-%m-%d %H:%M')
    
    # Build the report
    report = f"""
{'='*80}
    DAILY CYBERCRIME INTELLIGENCE BRIEF
    Generated: {report_date}
    Source: Automated Newspaper Agent (Real-time RSS Feeds)
    AI Model: Llama 3.2 (Local)
{'='*80}

📊 SUMMARY
   • Total articles scanned: {len(all_articles)}
   • Cybercrime incidents identified: {len(summaries)}
   • Analysis method: {'Llama 3.2 Local LLM' if LLM_AVAILABLE else 'Keyword-based'}
   • News sources: Times of India, The Hindu, NDTV, India Today, Hindustan Times, Indian Express

{'='*80}

🚨 KEY INCIDENTS

"""
    
    # Add each incident
    for idx, summary in enumerate(summaries, 1):
        report += f"""
{'─'*80}
INCIDENT #{idx}
{'─'*80}

📰 Headline:
   {summary['title']}

🔍 Intelligence Summary:
   {summary['summary']}

📌 Source: {summary['source']}
🔗 URL: {summary.get('url', 'N/A')}

"""
    
    # Add footer
    report += f"""
{'='*80}

📝 NOTES
   • This is an automated prototype using real-time RSS feeds and local Llama 3.2 model
   • Human verification recommended for high-priority incidents
   • For urgent matters, contact the Cybercrime Coordination Center
   • Report generated from live news sources at {report_date}

{'='*80}
End of Report
{'='*80}
"""
    
    return report


# Generate and display the final report
final_report = generate_intelligence_report(intelligence_summaries)
print(final_report)

# Optionally save to file
report_filename = f"cybercrime_brief_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(final_report)

print(f"\n💾 Report saved to: {report_filename}")
print("\n✅ Stage 4 (REPORT) complete: Intelligence brief generated")
print("\n🎉 AGENT CYCLE COMPLETE!")

## 🚀 Future Enhancements (Stage-2 and Beyond)

This Stage-1 prototype demonstrates the core agent pattern with real-time data. Here's how we can evolve it:

### **Immediate Next Steps (Stage-2)**
- **Automation**: Schedule the agent to run every 6 hours using cron jobs or cloud functions
- **More RSS Feeds**: Add regional news sources and specialized cybercrime publications
- **Better Classification**: Replace keyword matching with LLM-based relevance scoring
- **Sentiment Analysis**: Track public sentiment around cybercrime incidents
- **Deduplication**: Identify and merge duplicate stories from different sources

### **Medium-Term (Stage-3)**
- **Admin Dashboard**: Build a web interface for viewing reports and managing alerts
- **Alert System**: Send notifications for high-priority incidents (email, SMS, Slack)
- **Trend Detection**: Track recurring patterns and emerging threats over time
- **Multi-Source Integration**: Add social media monitoring (Twitter, Reddit)
- **Database Storage**: Store articles and summaries for historical analysis

### **Advanced (Stage-4 - Multi-Agent System)**
- **Specialist Agents**: Deploy separate agents for different cybercrime types
  - Agent 1: Financial fraud (UPI, cryptocurrency)
  - Agent 2: Social engineering (phishing, deepfakes)
  - Agent 3: Infrastructure attacks (ransomware, DDoS)
- **Coordinator Agent**: Synthesizes findings from all specialist agents
- **Vector Database**: Store and search historical intelligence reports
- **Predictive Analytics**: Use ML to forecast cybercrime trends

### **Production Deployment**
- **API Integration**: Connect with government databases and case management systems
- **Compliance**: Ensure data privacy and security standards (encryption, access logs)
- **Human-in-the-Loop**: Allow analysts to provide feedback and refine agent behavior
- **Scalability**: Deploy on cloud infrastructure (AWS, Azure, GCP) for 24/7 operation

---

### **Key Principle (Ed Donner Style)**

> *"Start simple, prove value, then scale."*

This notebook now uses real data and proves the concept works with live news sources. Now we can confidently add complexity where it matters most.

---

**Questions? Feedback?**  
This is a living prototype. Test it, break it, improve it. That's how great agents are built. 🛠️